# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize
import datetime
import cv2
from PIL import Image

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

C:\Users\DX\Anaconda3\envs\MaskRCNN\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# remember to add the step  after git python PythonAPI/setup.py build_ext install

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def get_width(xy):
    width = abs(xy[1] - xy[3])
    return width

def get_height(xy):
    height = abs(xy[0] - xy[2])
    return height

def get_area(xy):
    width = get_width(xy)
    height = get_height(xy)
    area = width * height
    return area

def get_biggest_box(xy_list):
    biggest_area = 0
    for i, xy in enumerate(xy_list):
        area = get_area(xy)
        biggest_area = area
        biggest_xy = xy
        ix = i
    return biggest_xy, ix

def overlay_box(image, xy): 
    position = (xy[1], xy[0])
    width = get_width(xy)
    height = get_height(xy)
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    rect = patches.Rectangle(position, 
                             width, 
                             height,
                             linewidth=1,
                             edgecolor='r',
                             facecolor='none')
    ax.add_patch(rect)
    plt.show()

In [ ]:
big_box, big_ix = get_biggest_box(r['rois'])
#make_box_mask(image, big_box)

In [ ]:
big_box=r['rois'][1]

In [ ]:
r['rois'].shape

In [ ]:
big_box=np.array([118,  76, 257, 277])

In [ ]:
def make_box_mask(image, xy):    
    target = image[xy[0]:xy[2], xy[1]:xy[3], :]
    img = np.zeros_like(image)
    img[xy[0]:xy[2], xy[1]:xy[3], :] = target
    plt.imshow(img)
    


In [ ]:
make_box_mask(image, big_box)

In [ ]:
def make_segmentation_mask(image, mask):
    img = image.copy()
    img[:,:,0] *= mask
    img[:,:,1] *= mask
    img[:,:,2] *= mask
    plt.imshow(img)
    plt.savefig('myfig.png')



In [ ]:
plt.axis('off')
plt.imshow(cropped)
plt.savefig('1.png')

In [ ]:
len (r['rois'])

def cropper(image,array_mask):
    basename='img'
    q=0
    suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    for i in range(len(array_mask)):
        
        cropped = image[r['rois'][q][0]:r['rois'][q][2],r['rois'][q][1]:r['rois'][q][3]]
        #f=cropped
        
        x=r['rois'][q][0]]  
        y=
        w=
        h= 
        
        
        plt.draw()
        plt.figure()
        plt.axis('off')
        plt.imshow(cropped)
        #suffix=str(q)
        #filename = "_".join([basename, suffix])
        #plt.savefig('result/{1}{0}.jpg'.format(i,image))
        q=q+1
        filename = "_".join([basename, suffix])
        plt.savefig('result/'+filename+'.jpg')
    

In [8]:
#with opencv
def cropper(array_mask, cv_image):
    basename='img'
    q=0
    suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    for i in range(len(array_mask)):
        
        #cropped = image[r['rois'][q][0]:r['rois'][q][2],r['rois'][q][1]:r['rois'][q][3]]
        #f=cropped
        
        p3=
        p2
        
        x=r['rois'][q][3]+5  
        y=r['rois'][q][2]+3
        w= r['rois'][q][1]-r['rois'][q][3]-10
        h= r['rois'][q][0]-r['rois'][q][2]-10
        y2=y+h
        x2=x+w
        #img=cv2.rectangle(cv_read,(x,y),(x+w,y+h),(0,0,255),1)
        roi_v= cv_image[y2:y,x2:x]
        height, width  = roi_v.shape[:2]
        if height > 40 and width > 30:
            filename = "_".join([basename, suffix])
            #cv2.imshow('show',cv_read)
            cv2.imwrite('result/'+filename+'.jpg',roi_v)
            continue
        q=q+1
        
        




        
        #if cv2.waitKey(25) & 0xFF==ord('q'):
         #   break
        
         
        
        #plt.draw()
        #plt.figure()
        #plt.axis('off')
        #plt.imshow(cropped)
        #suffix=str(q)
        #filename = "_".join([basename, suffix])
        #plt.savefig('result/{1}{0}.jpg'.format(i,image))
       
        #filename = "_".join([basename, suffix])
        #plt.savefig('result/'+filename+'.jpg')
    

In [ ]:
# Responsive padding

def cropper(array_mask, cv_image):
    basename='img'
    q=0
    suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    for i in range(len(array_mask)):
        
        #cropped = image[r['rois'][q][0]:r['rois'][q][2],r['rois'][q][1]:r['rois'][q][3]]
        #f=cropped
        
        p3=r['rois'][q][3]*.20
        p2=r['rois'][q][2]*.20
        
        x=r['rois'][q][3]+p3  
        y=r['rois'][q][2]+p2
        w= r['rois'][q][1]-r['rois'][q][3]-p3-5     # added more padding because the difference was  negligible
        h= r['rois'][q][0]-r['rois'][q][2]-p2-5
        y2=y+h
        x2=x+w
        #img=cv2.rectangle(cv_read,(x,y),(x+w,y+h),(0,0,255),1)
        roi_v= cv_image[y2:y,x2:x]
        height, width  = roi_v.shape[:2]
        if height > 40 and width > 30:
            filename = "_".join([basename, suffix])
            #cv2.imshow('show',cv_read)
            cv2.imwrite('result/'+filename+'.jpg',roi_v)
            continue
        q=q+1

mylist = os.listdir('data') # loading all the files 
for i in mylist:
    #image = skimage.io.imread('data/'+ i)
    cv_read=cv2.imread('data/'+ i)
    results = model.detect([image], verbose=1)
    r = results[0]
    cropper(image,r['rois'],cv_read)        
        

In [9]:
mylist = os.listdir('data') # loading all the files 
for i in mylist:
    #image = skimage.io.imread('data/'+ i)
    cv_read=cv2.imread('data/'+ i)
    results = model.detect([cv_read], verbose=1)
    r = results[0]
    cropper(r['rois'],cv_read)    

Processing 1 images
image                    shape: (110, 89, 3)          min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1006.00000
Processing 1 images
image                    shape: (244, 257, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  933.00000
Processing 1 images
image                    shape: (133, 44, 3)          min:    0.00000  max:  241.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  137.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (144, 100, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)

Processing 1 images
image                    shape: (178, 498, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  137.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (90, 87, 3)           min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  926.00000
Processing 1 images
image                    shape: (164, 172, 3)         min:    0.00000  max:  250.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  143.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  931.00000
Processing 1 images
image                    shape: (219, 375, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)

Processing 1 images
image                    shape: (91, 66, 3)           min:    0.00000  max:  245.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  139.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (98, 86, 3)           min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  146.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  968.00000
Processing 1 images
image                    shape: (106, 139, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (274, 239, 3)         min:    0.00000  max:  245.00000
molded_images            shape: (1, 1024, 1024, 3)

Processing 1 images
image                    shape: (111, 128, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  973.00000
Processing 1 images
image                    shape: (93, 59, 3)           min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (163, 78, 3)          min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  149.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (103, 58, 3)          min:    0.00000  max:  236.00000
molded_images            shape: (1, 1024, 1024, 3)

Processing 1 images
image                    shape: (84, 363, 3)          min:    0.00000  max:  244.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  128.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1024.00000
Processing 1 images
image                    shape: (153, 137, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  958.00000
Processing 1 images
image                    shape: (230, 263, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  969.00000
Processing 1 images
image                    shape: (160, 128, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)

In [46]:
#image = skimage.io.imread('data/500.jpg')
cv_read=cv2.imread('data/500.jpg')

results = model.detect([cv_read], verbose=1)
r = results[0]
#cropper(image,r['rois'],cv_read)     

Processing 1 images
image                    shape: (234, 280, 3)         min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max:  990.00000


In [ ]:
x=r['rois'][0][0]  
x

In [ ]:
y=['rois'][][1]
y

In [ ]:
w= r['rois'][q][3]-r['rois'][q][0]
h= r['rois'][q][2]-r['rois'][q][1]
roi_vehchile=cv_image[y:y+ h, x:x+ w]
filename = "_".join([basename, suffix])

In [ ]:
cvread=cv2.imread('data/500.jpg')
x=(r['rois'][3][3]+5)
y=(r['rois'][3][2]+3)
w= ((r['rois'][3][1]-r['rois'][3][3])-10)
h= ((r['rois'][3][0]-r['rois'][3][2])-10)
y2=y+h
x2=x+w
#img=cv2.rectangle(cv_read,(x,y),(x+w,y+h),(0,0,255),1)
read = cvread[y2:y,x2:x]
height, width = read.shape[:2]
print(height, width)

cv2.imshow('show',read)
cv2.waitKey()
cv2.destroyAllWindows()
    

    


67 35


In [ ]:
cv::flip(original_image,flip_image,1);
